<a href="https://colab.research.google.com/github/coordanalisis-sedatu/rezago/blob/main/rezago_hab_2018_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Evolución del rezago habitacional 2018-2022


In [ ]:
##Borrar datos del entorno
rm(list=ls())

In [ ]:
##Crear folders de almacenamiento
dir.create("microdatos", showWarnings = F)


In [ ]:
#Paquetería
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse,srvyr,janitor)
font_add_google("Poppins", "pop")
showtext_auto()


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘snakecase’



janitor installed



In [ ]:
#Función para descargar y unzip
download_and_unzip <- function(url, dir) {
  # Crear directorio si no existe
  if (!dir.exists(dir)) {
    dir.create(dir, showWarnings = FALSE)
  }

  # Crear el archivo destino
  destfile <- paste0(dir, "/", basename(url))

  # Descarga
  download.file(url, destfile)

  # Unzip
  unzip(destfile, exdir = dir)
}


In [ ]:
# Utiliza la función para descargar datos 2022 y 2018 de ENIGH
download_and_unzip("https://www.inegi.org.mx/contenidos/programas/enigh/nc/2022/microdatos/enigh2022_ns_viviendas_csv.zip", "microdatos")
download_and_unzip("https://www.inegi.org.mx/contenidos/programas/enigh/nc/2018/microdatos/enigh2018_ns_viviendas_csv.zip", "microdatos_18")

In [ ]:
# Carga y limpia el conjunto de datos de viviendas para el año 2022
viviendas_22 <- read.csv("microdatos/viviendas.csv") %>%
  janitor::clean_names() %>%  #Poner las variables en minusculas y quitar las variables
  mutate(
    mat_pared = as.numeric(mat_pared),
    mat_pisos = as.numeric(mat_pisos),
    mat_techos = as.numeric(mat_techos),
    rezago = case_when(
      (tot_resid / num_cuarto) > 2.5 |   # Hacinamiento
        mat_pared %in% c(1, 2, 3, 4, 5, 6) |
        mat_pisos %in% c(1) |
        mat_techos %in% c(1, 2, 3, 4, 6, 7, 9) |
        excusado == 2 ~ "En rezago",
      TRUE ~ "Fuera de rezago"
    ),
    vivi_autoprod = case_when(
      (tenencia %in% c(3, 4) & tipo_adqui %in% c(2, 3)) ~ "Sí",
      TRUE ~ "No"
    ),
    hacinamiento = tot_resid / num_cuarto,
    hacinamiento = ifelse(hacinamiento > 2.5, 1, 0),
    folioviv = as.character(folioviv),
    # Clave de entidad federativa
    cve_ent = case_when(
      nchar(folioviv) == 10 ~ as.character(substr(folioviv, 1, 2)), #1 y 2 clave de Entidad Federativa
      nchar(folioviv) == 9 ~ as.character(paste0("0", substr(folioviv, 1, 1))),
      TRUE ~ as.character(folioviv)
    ),
    # Variable para obtener el total de residentes que se encuentran en viviendas autoproducidas.
      tot_resid_autoprod = case_when(
      vivi_autoprod == "Sí" ~ tot_resid,
      TRUE ~ 0
    ),
    # Variable para obtener el total de hogares que se encuentran en viviendas autoproducidas.
    tot_hog_autoprod = case_when(
      vivi_autoprod == "Sí" ~ tot_hog,
      TRUE ~ 0
    ),
    # Nombre de cada entidad
    nom_ent = case_when(
      cve_ent == "01" ~ "Aguascalientes",
      cve_ent == "02" ~ "Baja California",
      cve_ent == "03" ~ "Baja California Sur",
      cve_ent == "04" ~ "Campeche",
      cve_ent == "05" ~ "Coahuila de Zaragoza",
      cve_ent == "06" ~ "Colima",
      cve_ent == "07" ~ "Chiapas",
      cve_ent == "08" ~ "Chihuahua",
      cve_ent == "09" ~ "Ciudad de México",
      cve_ent == "10" ~ "Durango",
      cve_ent == "11" ~ "Guanajuato",
      cve_ent == "12" ~ "Guerrero",
      cve_ent == "13" ~ "Hidalgo",
      cve_ent == "14" ~ "Jalisco",
      cve_ent == "15" ~ "México",
      cve_ent == "16" ~ "Michoacán de Ocampo",
      cve_ent == "17" ~ "Morelos",
      cve_ent == "18" ~ "Nayarit",
      cve_ent == "19" ~ "Nuevo León",
      cve_ent == "20" ~ "Oaxaca",
      cve_ent == "21" ~ "Puebla",
      cve_ent == "22" ~ "Querétaro",
      cve_ent == "23" ~ "Quintana Roo",
      cve_ent == "24" ~ "San Luis Potosí",
      cve_ent == "25" ~ "Sinaloa",
      cve_ent == "26" ~ "Sonora",
      cve_ent == "27" ~ "Tabasco",
      cve_ent == "28" ~ "Tamaulipas",
      cve_ent == "29" ~ "Tlaxcala",
      cve_ent == "30" ~ "Veracruz de Ignacio de la Llave",
      cve_ent == "31" ~ "Yucatán",
      cve_ent == "32" ~ "Zacatecas",
      TRUE ~ "No identificado"
    )
  )

Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `mat_pisos = as.numeric(mat_pisos)`.
Caused by warning:
! NAs introduced by coercion”


In [ ]:
# Carga y limpia el conjunto de datos de viviendas para el año 2018
viviendas_18 <- read.csv("microdatos_18/viviendas.csv") %>%
  janitor::clean_names() %>%  #Poner las variables en minusculas y quitar las variables
  mutate(
    mat_pared = as.numeric(mat_pared),
    mat_pisos = as.numeric(mat_pisos),
    mat_techos = as.numeric(mat_techos),
    rezago = case_when(
      (tot_resid / num_cuarto) > 2.5 |   # Hacinamiento
        mat_pared %in% c(1, 2, 3, 4, 5, 6) |
        mat_pisos %in% c(1) |
        mat_techos %in% c(1, 2, 3, 4, 6, 7, 9) |
        excusado == 2 ~ "En rezago",
      TRUE ~ "Fuera de rezago"
    ),
    vivi_autoprod = case_when(
      (tenencia %in% c(3, 4) & tipo_adqui %in% c(2, 3)) ~ "Sí",
      TRUE ~ "No"
    ),
    hacinamiento = tot_resid / num_cuarto,
    hacinamiento = ifelse(hacinamiento > 2.5, 1, 0),
    folioviv = as.character(folioviv),
    # Clave de entidad federativa
    cve_ent = case_when(
      nchar(folioviv) == 10 ~ as.character(substr(folioviv, 1, 2)), #1 y 2 clave de Entidad Federativa
      nchar(folioviv) == 9 ~ as.character(paste0("0", substr(folioviv, 1, 1))),
      TRUE ~ as.character(folioviv)
    ),
    # Variable para obtener el total de residentes que se encuentran en viviendas autoproducidas.
      tot_resid_autoprod = case_when(
      vivi_autoprod == "Sí" ~ tot_resid,
      TRUE ~ 0
    ),
    # Variable para obtener el total de hogares que se encuentran en viviendas autoproducidas.
    tot_hog_autoprod = case_when(
      vivi_autoprod == "Sí" ~ tot_hog,
      TRUE ~ 0
    ),
    # Nombre de cada entidad
    nom_ent = case_when(
      cve_ent == "01" ~ "Aguascalientes",
      cve_ent == "02" ~ "Baja California",
      cve_ent == "03" ~ "Baja California Sur",
      cve_ent == "04" ~ "Campeche",
      cve_ent == "05" ~ "Coahuila de Zaragoza",
      cve_ent == "06" ~ "Colima",
      cve_ent == "07" ~ "Chiapas",
      cve_ent == "08" ~ "Chihuahua",
      cve_ent == "09" ~ "Ciudad de México",
      cve_ent == "10" ~ "Durango",
      cve_ent == "11" ~ "Guanajuato",
      cve_ent == "12" ~ "Guerrero",
      cve_ent == "13" ~ "Hidalgo",
      cve_ent == "14" ~ "Jalisco",
      cve_ent == "15" ~ "México",
      cve_ent == "16" ~ "Michoacán de Ocampo",
      cve_ent == "17" ~ "Morelos",
      cve_ent == "18" ~ "Nayarit",
      cve_ent == "19" ~ "Nuevo León",
      cve_ent == "20" ~ "Oaxaca",
      cve_ent == "21" ~ "Puebla",
      cve_ent == "22" ~ "Querétaro",
      cve_ent == "23" ~ "Quintana Roo",
      cve_ent == "24" ~ "San Luis Potosí",
      cve_ent == "25" ~ "Sinaloa",
      cve_ent == "26" ~ "Sonora",
      cve_ent == "27" ~ "Tabasco",
      cve_ent == "28" ~ "Tamaulipas",
      cve_ent == "29" ~ "Tlaxcala",
      cve_ent == "30" ~ "Veracruz de Ignacio de la Llave",
      cve_ent == "31" ~ "Yucatán",
      cve_ent == "32" ~ "Zacatecas",
      TRUE ~ "No identificado"
    )
  )

Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `mat_pisos = as.numeric(mat_pisos)`.
Caused by warning:
! NAs introduced by coercion”


In [ ]:
#estructura general de la tabla
str(viviendas_18)

'data.frame':	73405 obs. of  71 variables:
 $ folioviv          : chr  "100013601" "100013602" "100013603" "100013604" ...
 $ tipo_viv          : chr  "1" "1" "1" "1" ...
 $ mat_pared         : num  8 8 8 8 8 8 8 8 8 8 ...
 $ mat_techos        : num  10 10 10 10 10 10 10 10 10 10 ...
 $ mat_pisos         : num  3 3 3 3 3 3 3 3 2 3 ...
 $ antiguedad        : int  42 40 50 45 40 5 25 30 1 30 ...
 $ antigua_ne        : int  NA NA NA NA NA NA NA NA NA NA ...
 $ cocina            : int  1 1 1 1 1 1 1 1 2 1 ...
 $ cocina_dor        : int  2 2 2 2 2 2 2 2 NA 2 ...
 $ cuart_dorm        : int  4 5 2 2 3 3 3 1 2 2 ...
 $ num_cuarto        : int  6 6 7 5 4 5 5 3 3 8 ...
 $ disp_agua         : int  1 1 1 1 1 1 1 1 1 1 ...
 $ dotac_agua        : int  1 1 1 1 4 1 2 1 2 1 ...
 $ excusado          : int  1 1 1 1 1 1 1 1 1 1 ...
 $ uso_compar        : int  2 2 2 2 2 2 2 2 2 2 ...
 $ sanit_agua        : int  1 1 1 1 1 1 1 1 1 1 ...
 $ biodigest         : int  2 2 2 2 2 2 2 2 2 2 ...
 $ bano_comp        

In [ ]:
#estructura general de la tabla
str(viviendas_22)

'data.frame':	88823 obs. of  71 variables:
 $ folioviv          : chr  "100005002" "100005003" "100005004" "100012002" ...
 $ tipo_viv          : chr  "1" "1" "1" "1" ...
 $ mat_pared         : num  8 8 8 8 8 8 8 8 8 8 ...
 $ mat_techos        : num  10 10 10 10 10 10 10 10 10 10 ...
 $ mat_pisos         : num  3 3 3 3 3 3 3 3 3 3 ...
 $ antiguedad        : int  30 15 25 50 45 25 48 36 35 1 ...
 $ antigua_ne        : int  NA NA NA NA NA NA NA NA NA NA ...
 $ cocina            : int  1 1 1 1 1 1 1 1 1 1 ...
 $ cocina_dor        : int  2 2 2 2 2 2 2 2 2 2 ...
 $ cuart_dorm        : int  2 3 3 4 3 2 5 4 3 3 ...
 $ num_cuarto        : int  4 5 7 6 4 4 6 7 4 6 ...
 $ disp_agua         : int  1 1 1 1 1 1 1 1 1 1 ...
 $ dotac_agua        : int  1 1 1 1 1 2 1 1 1 1 ...
 $ excusado          : int  1 1 1 1 1 1 1 1 1 1 ...
 $ uso_compar        : int  2 2 2 2 2 2 2 2 2 2 ...
 $ sanit_agua        : int  1 1 1 1 1 1 1 1 1 1 ...
 $ biodigest         : int  2 2 2 2 2 2 2 2 2 2 ...
 $ bano_comp        

In [ ]:
#Definir diseño muestral para ambos años
dm22<-viviendas_22%>%
  #Diseño muestral
  as_survey(weights=factor, strata=est_dis, ids=upm)

dm18<-viviendas_18%>%
  #Diseño muestral
  as_survey(weights=factor, strata=est_dis, ids=upm)


In [ ]:
# ------------------------------------------------ Mostrar las viviendas que se encuentran en condición de rezago dentro del diseño muestral 2022
dm22 %>%
  group_by(rezago) %>%
  summarise(viviendas = survey_total(vartype = "cv", level = 0.95)) %>%
  janitor::adorn_totals()

rezago,viviendas,viviendas_cv
<chr>,<dbl>,<dbl>
En rezago,8977409,0.011080097
Fuera de rezago,27991462,0.004748287
Total,36968871,0.015828384


In [ ]:
# ------------------------------------------------ Mostrar las viviendas que se encuentran en condición de rezago dentro del diseño muestral 2018
dm18 %>%
  group_by(rezago) %>%
  summarise(viviendas = survey_total(vartype = "cv", level = 0.95)) %>%
  janitor::adorn_totals()

rezago,viviendas,viviendas_cv
<chr>,<dbl>,<dbl>
En rezago,9238897,0.012202672
Fuera de rezago,24575235,0.005636817
Total,33814132,0.017839490
